In [ ]:
# !pip install diffusers transformers accelerate safetensors --upgrade

In [ ]:
import torch
from PIL import Image, ImageFilter

from diffusers import AutoPipelineForImage2Image, AutoPipelineForText2Image
from diffusers import AutoPipelineForInpainting, ControlNetModel
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers import StableDiffusionPipeline as SDPipeline
from diffusers import StableDiffusionControlNetPipeline as SDCNPipeline
from diffusers import StableDiffusionImageVariationPipeline as SDIVPipeline
from diffusers import StableDiffusionXLImg2ImgPipeline as SDXLI2IPipeline

from torchvision import transforms

### [Stable Diffusion](https://huggingface.co/stabilityai/stable-diffusion-2)

In [ ]:
pipe = SDPipeline.from_pretrained(
  "CompVis/stable-diffusion-v1-4",
  #"runwayml/stable-diffusion-v1-5",
  #"stabilityai/stable-diffusion-2-1",
  safety_checker=None,
  torch_dtype=torch.float16
).to("cuda")

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
out = pipe("an egg walking around on the moon", num_inference_steps=32)
display(out["images"][0])

### Control Net

#### [Scribble](https://huggingface.co/lllyasviel/sd-controlnet-scribble)

<img src="../imgs/scribble_00.jpg" width="200px">
<img src="../imgs/scribble_01.jpg" width="200px">

#### [Depth](https://huggingface.co/lllyasviel/sd-controlnet-depth)

<img src="../imgs/depth_00.jpg" width="200px">
<img src="../imgs/depth_01.jpg" width="200px">

In [ ]:
controlnet = ControlNetModel.from_pretrained(
  "lllyasviel/sd-controlnet-scribble",
  # "lllyasviel/sd-controlnet-depth",
  torch_dtype=torch.float16
)

pipe = SDCNPipeline.from_pretrained(
  "runwayml/stable-diffusion-v1-5",
  controlnet=controlnet,
  safety_checker=None,
  torch_dtype=torch.float16
).to("cuda")

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
im = Image.open("../imgs/scribble_00.jpg").convert("RGB")
out = pipe("person", im, num_inference_steps=20)
display(out["images"][0])

### [Turbo](https://huggingface.co/stabilityai/sd-turbo)

https://www.reddit.com/r/StableDiffusion/comments/1e6cq09/physical_interfaces_realtime_img2img_diffusion/

In [ ]:
pipe = AutoPipelineForText2Image.from_pretrained(
  "stabilityai/sd-turbo",
  torch_dtype=torch.float16,
  variant="fp16"
).to("cuda")

In [ ]:
prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."
out = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0, num_images_per_prompt=4)
display(out["images"][0])

In [ ]:
pipe = AutoPipelineForImage2Image.from_pretrained(
  "stabilityai/sd-turbo",
  torch_dtype=torch.float16,
  variant="fp16"
).to("cuda")

In [ ]:
im = Image.open("./imgs/scribble_01.jpg").convert("RGB")
prompt = "cat wizard, gandalf, lord of the rings, detailed, fantasy, cute, adorable, Pixar, Disney, 8k"
out = pipe(prompt, image=im, num_inference_steps=2, strength=0.5, guidance_scale=0.0)
display(out["images"][0])

### [Inpainting](https://huggingface.co/docs/diffusers/en/using-diffusers/inpaint)

In [ ]:
pipe = AutoPipelineForInpainting.from_pretrained(
  #"runwayml/stable-diffusion-inpainting",
  #"stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting",
  torch_dtype=torch.float16,
  variant="fp16"
).to("cuda")

In [ ]:
im = Image.open("./imgs/landscape.jpg")
mask = Image.open("./imgs/landscape_mask.jpg").filter(ImageFilter.GaussianBlur((16,0)))
iw,ih = im.size

prompt = "bob ross oil painting of nature landscape with trees, detailed brushstrokes, mountains, lake, rocks"

output = pipe(prompt=prompt, image=im, mask_image=mask, num_inference_steps=24, width=iw, height=ih)
display(output.images[0])

### [Image Variation](https://huggingface.co/lambdalabs/sd-image-variations-diffusers)

In [ ]:
pipe = SDIVPipeline.from_pretrained(
  "lambdalabs/sd-image-variations-diffusers",
  revision="v2.0",
  safety_checker=None
).to("cuda")

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
im = Image.open("../imgs/scribble_01.jpg").convert("RGB")

preprocessor = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize(
    (224, 224),
    interpolation=transforms.InterpolationMode.BICUBIC,
    antialias=False),
  transforms.Normalize(
    [0.48145466, 0.4578275, 0.40821073],
    [0.26862954, 0.26130258, 0.27577711]),
])

inp = preprocessor(im).to("cuda").unsqueeze(0)

In [ ]:
out = pipe(inp, guidance_scale=3, num_inference_steps=64)
display(out["images"][0])

### [Refiner](https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0)

In [ ]:
pipe = SDXLI2IPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-refiner-1.0",
  torch_dtype=torch.float16,
  variant="fp16",
  use_safetensors=True,
  add_watermarker=False
).to("cuda")

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
im = Image.open("../imgs/scribble_01.jpg").convert("RGB")
prompt = "a photo of a man"
out = pipe(prompt, image=im, num_inference_steps=64)
display(out["images"][0])